In [1]:
import os
import pyspark.mllib
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import pyspark

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from IPython.display import display, HTML

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# initialize streaming context
conf = SparkConf().setMaster("local[2]").setAppName("AutoRegression")

sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("spark play").getOrCreate()

- Attached file auto_mpg_original.csv contains a set of data on automobile characteristics and fuel consumption.
- File auto_mpg_description.csv contains the description of the data.
- Find all null values in all numerical columns. Replace nulls, if any, with average values for respective columns using Spark Data Frame API.
- Import data into Spark.

In [3]:
path = "file:////Users/swaite/Stirling/CSIE-63/assignment-7/data/auto_mpg_original-1.csv"
mapped_auto_rdd = sc.textFile(path, use_unicode=False) \
                    .map(lambda x: x.split(","))\
                    .filter(lambda x: (x[0] > 0) & (x[3] > 0))\
                    .map(lambda x: Row(
                        mpg=x[0],
                        cylinders=x[1],
                        displacement=x[2],
                        horsepower=x[3],
                        weight=x[4],
                        acceleration=x[5],
                        model_year=x[6],
                        origin=x[7],
                        car_name=x[8]
                    ))\
                    .toDF()
        
print(mapped_auto_rdd.show(5))

+------------+---------+---------+------------+----------+----------+---+------+------+
|acceleration| car_name|cylinders|displacement|horsepower|model_year|mpg|origin|weight|
+------------+---------+---------+------------+----------+----------+---+------+------+
|          12|chevrolet|        8|         307|       130|        70| 18|     1|  3504|
|        11.5|    buick|        8|         350|       165|        70| 15|     1|  3693|
|          11| plymouth|        8|         318|       150|        70| 18|     1|  3436|
|          12|      amc|        8|         304|       150|        70| 16|     1|  3433|
|        10.5|     ford|        8|         302|       140|        70| 17|     1|  3449|
+------------+---------+---------+------------+----------+----------+---+------+------+
only showing top 5 rows

None


- Randomly select 10-20% of you data for testing and use remaining data for training.

In [4]:
training_data, testing_data = mapped_auto_rdd.randomSplit([.8, .2], seed=1234)
print("training_data count: " + str(training_data.count()))
print("testing_data count: " + str(testing_data.count()))
print("mapped_auto_rdd count: " + str(mapped_auto_rdd.count()))

training_data count: 334
testing_data count: 72
mapped_auto_rdd count: 406
